**MODEL BUILDING:-**

1. Importing The Model Building Libraries

In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

2. Loading The Model

In [27]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/IBM - PROJECT/Dataset/INTELLIGENT/body/training'
valid_path = '/content/drive/MyDrive/IBM - PROJECT/Dataset/INTELLIGENT/body/validation'

In [28]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

3. Adding Flatten Layer

In [29]:
for layer in vgg16.layers:
    layer.trainable = False

In [30]:
folders = glob('/content/drive/MyDrive/IBM - PROJECT/Dataset/INTELLIGENT/body/training/*')

In [31]:
folders

['/content/drive/MyDrive/IBM - PROJECT/Dataset/INTELLIGENT/body/training/Untitled.ipynb',
 '/content/drive/MyDrive/IBM - PROJECT/Dataset/INTELLIGENT/body/training/02-side',
 '/content/drive/MyDrive/IBM - PROJECT/Dataset/INTELLIGENT/body/training/00-front',
 '/content/drive/MyDrive/IBM - PROJECT/Dataset/INTELLIGENT/body/training/01-rear']

In [32]:
x = Flatten()(vgg16.output)

In [33]:
len(folders)

4

4. Adding Output Layer

In [34]:
prediction = Dense(len(folders), activation='softmax')(x)

5. Creating A Model Object

In [35]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [36]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

6. Configure The Learning Process

In [37]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

7. Train The Model

In [49]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
98/98 [==============================] - ETA: 0s - loss: 0.6865 - accuracy: 0.7344

InvalidArgumentError: ignored

8. Save The Model

In [18]:
from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/IBM - PROJECT/Dataset/INTELLIGENT/body.h5')

9. Test The Model

In [19]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [20]:
model = load_model('/content/drive/MyDrive/IBM - PROJECT/Dataset/INTELLIGENT/body.h5')

In [21]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["front","rear","side"]
  preds = label[np.argmax(prediction)]
  return preds

In [22]:
import numpy as np

In [23]:
data = "/content/drive/MyDrive/IBM - PROJECT/Dataset/INTELLIGENT/body/training/00-front/0005.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 1s/step
rear
